## 8 어텐션
- seq2seq는 2개의 RNN을 연결 하여 하나의 시계열 데이터를 다른 시계열 데이터로 변환한다.

## 8.1 어텐션의 구조
- 어텐션 메커니즘을 통해 seq2seq는 (인간처럼)필요한 정보에만 '주목'할 수 있게 된다.
- 기존 seq2seq에 존재했던 문제점을 해결할 수 있다.

#### 8.1.1 seq2seq의 문제점
- seq2seq 는 Encoder가 시계열 데이터를 인코딩 한다, 이후 인코딩된 정보를 Decoder에 전달한다.
- 그런데 Encoder의 출력은 항상 '고정 길이 벡터'였다
- 하지만 이 벡터 길이의 고정 때문에 문장에 길이에 관계 없어 항상 같은 길이의 벡터로 변환한다는 뜻이다.
- 아무리 긴 문장이라도 고정된 길이의 벡터로 변환된다.
- 이러한 Encoder의 문제점을 개선하여 성능을 향상시킬 수 있다.

#### 8.1.2 Encoder 개선
- seq2seq에서는 LSTM 계층의 마지막 은닉 상태만 Decoder에 전달 했다.
- 그러나 Encoder 출력의 길이는 입력 문장에 따라 바꿔주는 것이 좋다.
- 이점이 Encoder의 개선 포인트이다.

![image](https://github.com/choibigo/Study/assets/38881179/69e73189-1caa-49c5-afc9-5af56f839d22)

- 각 LSTM의 Output(각 hidden  state)을 모두 이용하면 입력된 단어와 같은 수의 벡터를 얻을 수 있다. (문자의 길이에 따라서 Encoder의 최종 출력의 크기가 가변적이다.)
- 이를 통해 Encoder는 '하나의 고정 길이 벡터'라는 제약을 극복할 수 있다.
- 시각 별 LSTM 계층의 hidden state에는 직전에 입력된 단어에 대한 정보가 많이 담겨 있다.
- '고양이'를 입력한 LSTM 계층의 output은 '고양이'라는 단어에 영향을 가장 크게 받는다.
- 따라서 이 hidden state는 '고양이'에 대한 '성분'이 많이 들어간 벡터라고 생각할 수 있다.

<img width="671" alt="fig 8-3" src="https://github.com/choibigo/Study/assets/38881179/95b5b914-4579-448d-a429-e79d2cc0c974">

- '고양이' 벡터에는 정확히 '나', '는', '고양이' 에 대한 정보 모두가 담겨 있다.
- 만약 고양이 주변의 정보를 균형있게 담고 싶다면 양방향 LSTM을 사용하는 것이 효과 적이다.
- 은닉 상태를 시각 마다 꺼냈을 뿐이지만, Encoder는 입력 문장의 길이에 비례한 정보를 인코딩할 수 있게 됐다.

#### 8.1.3 Decoder 개선 (1)
- Encoder는 각 단어에 대응하는 LSTM 계층의 은닉 상태 벡터를 hs로 모아 출력한다.
- 그리고 hs가 Decoder에 전달되어 시계열 변환이 이뤄진다.
- 각 LSTM의 hidden state가 담겨 있는 hs를 모두 활용할 수 있도록 Decoder를 수정해야 한다.
- 입력과 출력의 단어 중 어떤 단어끼리 서로 관련되어 있는가 를 학습 시킬 수 있다면 성능이 향상될 것이다.
- 따라서 학습의 목표는 '도착어 단어'에 대응하는 '출발어 단어'의 정보를 골라내는 것이다.
- 그리고 이 정보를 이용하여 번역을 수행하는 것이다.
- **필요한 정보에만 주목하여 그 정보로부터 시계열 반환을 수행하는 것이 목표이다.**

<img width="681" alt="fig 8-6" src="https://github.com/choibigo/Study/assets/38881179/45bb60e5-f44c-4af0-bd2c-8871dc28793f">

- Decoder 의 LSTM의 hidden state가 Afffine 계층으로 입력되기 전 '어떤 계산'을 수행한다.
- 이 어떤 계산의 입력은 2가지로 하나는 Encoder로부터 받는 hs이고 다른 하나는 시각별 LSTM 계층의 Hidden State이다.
- 이 연산의 의미는 Decoder의 각 시각의 입력과 Decoder의 각 hidden state(최초 입력 단어)간의 대응 관계를 파악하는 것이다.
- Decoder가 I를 출려할 때, hs(Encoder Output)에서 '나'에 해당하는 hidden state를 선택하는 것이다.
- 이러한 선택 작업을 '어떤 계산'을 통해서 하는 것이다.
- '선택한다'라는 작업을 미분 가능한 연산으로 대체 해야한다.
- '하나를 선택'하는 게 아니라, '모든 것을 선택'한다는 것이다.
- 각 단어의 중요도(기여도)를 나타내는 '가중치'를 별도로 계산하는 것이다.

<img width="627" alt="fig 8-7" src="https://github.com/choibigo/Study/assets/38881179/a78d99da-7aa6-4a61-a8f0-69b0e32eb5d7">

- Encoder의 output Hidden State들 에서 입력 단어에 대하여 그것이 얼마나 중요한지 구한다.
- 각 단어의 중요도를 나타내는 '가중치' a를 이용한다.
- a는 확률 분포 처럼 각 원소가 0~1 사이의 스칼라 값이며, 모든 원소의 총합은 1 이된다.
- 각 단어의 중요도를 나타내는 가중치 a와 단어의 벡터 hs로 부터 가중치 합을 구하여, 우리가 원하는 단어간 관게에 대한 벡터를 얻는다.

<img width="640" alt="fig 8-8" src="https://github.com/choibigo/Study/assets/38881179/152ec8d1-7452-4aa4-9629-5cf0f0993cb4">

- 그 결과를 '맥락 벡터'라고 부르며 c라고 표기 한다.
- 나에 대응하는 가중치가 0.8이다, 이것이 의미하는 바는 맥락 벡터 c에는 '나'라는 벡터의 성분이 많이 포함되어 있다느 ㄴ것이다.
- '나' 벡터를 '선택'하는 작업을 이 가중합으로 대체 하고 있다, '나'에 대응하는 가중치가 1이고 그 외에는 0 이라면, '나'벡터를 '선택'했다 라고 해석할 수

In [20]:
import numpy as np

np.random.seed(100)

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5, 1).repeat(4, axis=1)
#  hs각각에 원소에 weight를 곱해주기 위함,  repeat를 통해서 각 hs에 대응하는 가중치 원소를 만든다.
# 그냥 a(weight)를 곱해도 같은 연산을 수행할 수 있으나 repeat()를 사용하지 않으면 브로드캐스트 연산이 자동으로 일어나며 원소의 복사가 반복된다.

c = np.sum(hs*ar, axis=0)
# 원소별 곱을 통해 hs는 각 원소에 대응하는 weight만큼의 가중치를 얻는다, 이후 row방향으로 모두 더한다.
# 만약 1개만 1이고 나머지가 0이라면 1인 원소에 해당하는 hs가 선택 된 것이라 생각할 수 있다.


[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]
(4,)


<img width="490" alt="fig 8-10" src="https://github.com/choibigo/Study/assets/38881179/1b6b0240-5dcc-4392-855e-c37bd0523638">

In [ ]:
# Batch를 적용한 가중합 구하기

N, T, H = 10, 5, 4
hs = np.random.randn(10, 5, 4)
a = np.random.rand(N, T)
ar = a.reshape(N, T, 1).repeate(H, axis=2)

t = hs*ar # t = 10x5x4
c = np.sum(t, axis=1) # c = 10x4

<img width="268" alt="fig 8-11" src="https://github.com/choibigo/Study/assets/38881179/d57d80ac-3d7e-4710-b2c5-557fdd2c54a9">

#### 역전파
- sum의 역전파는 repeat이고 repeat 역전파는 sumdlek.

In [ ]:
class WeeightSUM:
    def __init__(self):
        self.params = []
        self.grads = []
        self.cache = None
    
    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshpae(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        
        dt = dc.reshape(N, 1, H).repeat(T, axis=1) # sum의 역전파
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2) # repeat의 역전파

        return dhs, da

#### 8.1.4 Decoder 개선 (2)
- 각 단어의 중요도를 나타내는 가중치 a가 있다면, 가중치 합을 이용해 '맥락 벡터'를 얻을 수 있다.
- 이 가중치 'a'를 어떻게 구할까?
- 일단 첫번째 Decoder의 첫 LSTM 계층이 은닉 상태 벡터를 출력할 때까지의 처리부터 알아봐야한다.

<img width="449" alt="fig 8-12" src="https://github.com/choibigo/Study/assets/38881179/feeada49-9479-498d-9119-e12425520f65">

- Decoder의 첫 LSTM 계층의 입력으로 hs의 마지막 index (= 마지막 hidden state) 벡터가 입력된다.
- 이후 LSTM은 hidden state h를 출력한다.
- hs와 h가 얼마나 '비슷한가'를 수치로 나타내야한다.
- '내적'을 이용하여 두 벡터의 '유사도'를 표현할 수 있다.

<img width="471" alt="fig 8-13" src="https://github.com/choibigo/Study/assets/38881179/283d5059-8b1f-4249-a5e5-e3344c3d9806">

- 벡터의 내적을 이용해 h와 hs의 각각의 벡터의 유사도를 구할 수 있다.
- 이후 softmax를 이용하여 정규화할 수 있다.

<img width="394" alt="fig 8-14" src="https://github.com/choibigo/Study/assets/38881179/d8a36827-766c-4a53-96f2-443c33ac4ee2">

<img width="300" alt="fig 8-15" src="https://github.com/choibigo/Study/assets/38881179/86d0ac97-99d4-4137-a222-60dfc0eedff4">

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Softmax

class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None
    
    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H).repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a
    
    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape
        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeate(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh

#### 8.1.5 Decoder 개선 (3)
- Attension Weight 계층과 Weight Sum 계층을 통해 h에 대응 하는 hs를 선택할 수 있게 됬다.
- Attension Weight : 각 LSTM의 출력이 어떤 hs에 주목해야 할지 0~1 사이의 weight를 구한다.
- Weight Sum : 구한 Weight를 통해서 실제로 hs와 연산을 통해 집중해야 하는 hs를 구한다.

<img width="656" alt="fig 8-16" src="https://github.com/choibigo/Study/assets/38881179/fe4782aa-3f58-4cf7-b7b9-98e18fa16360">

- Attension Weight 계층은 Encoder가 출력하는 각 단어 벡터 hs에 주목하여 해당 단어의 가중치 a를 구한다.
- 이후 weight sum 계층이 a와 hs의 가중합을 구하고 그 결과 맥락 벡터 c를 출력한다.

![image](https://github.com/choibigo/Study/assets/38881179/47167466-100f-4c37-a755-ffac08a90a75)

- 이러한 계산을 수행하는 계층을 Attention 계층이라고 부른다.

<img width="613" alt="fig 8-17" src="https://github.com/choibigo/Study/assets/38881179/e9bb748f-f666-459a-ae8c-f3a74c3ed9d9">

- Encoder가 건네주는 정보 hs에서 중요한 원소에 주목하여 그것을 바탕으로 맥락 벡터를 구해 위쪽 계층으로 전파하는 계층이다.

In [ ]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeeightSUM()
        self.attention_weight = None
    
    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        
        return out
    
    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh

<img width="673" alt="fig 8-18" src="https://github.com/choibigo/Study/assets/38881179/97ecf581-eb66-4b5a-ba7f-77b9b0ae7ad5">

- 각 시각의 Attention 계층에는 Encoder의 출력인 **hs**과 각 LSTM의 출력이 입력된다.
- 이후 결과를 Affine 계층에 입력한다.

<img width="673" alt="fig 8-19" src="https://github.com/choibigo/Study/assets/38881179/01611f7b-78cd-4ffd-b37d-f70629a26312">

- 기존 Decoder에 Attention Layer가 추가 됬다.
- 이 Attention Layer를 통해 입력 데이터가 어떤 hidden state에 주목해야할지 알수 있게 됬다.